In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import csv

In [3]:
#daraz_url = 'https://www.daraz.com.bd/catalog/?page={page_number}&q={search%20word}'

In [4]:
path = "C:\\Users\\tasnu\\Downloads\\Web Scraping Daraz\\chromedriver.exe"

option = Options()
option.add_argument("--headless")
service = Service(path)
chrome_driver = webdriver.Chrome(service=service, options=option)

In [5]:
def get_product_links(search_keyword, page_number, driver):
    
    search_keyword = search_keyword.replace(' ', '%20')
    daraz_url = f'https://www.daraz.com.bd/catalog/?page={page_number}&q={search_keyword}'
    driver.get(daraz_url)
    elements = driver.find_elements("css selector", 'div._17mcb a[href]')

    links = []
    for element in elements:
        link = element.get_attribute('href')
        links.append(link)

    for i in range(len(links) - 1, 0, -1):
        if i % 2 != 0:
            del links[i]
    
    return links

In [6]:
def get_product_details(driver, link):
    
    driver.get(link)
    
    try:
        product_name = driver.find_element("css selector", 'h1.pdp-mod-product-badge-title').text
    except NoSuchElementException:
        product_name = None

    try:
        brand_name = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[1]').text
    except NoSuchElementException:
        brand_name = None

    try:
        price = driver.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span').text
    except NoSuchElementException:
        price = None

    try:
        seller = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[1]').text
    except NoSuchElementException:
        seller = None

    try:
        seller_rating = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[1]/div[2]').text
    except NoSuchElementException:
        seller_rating = None

    try:
        seller_shipping = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]').text
    except NoSuchElementException:
        seller_shipping = None

    try:
        seller_chat = driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[3]/div[2]').text
    except NoSuchElementException:
        seller_chat = None

    try:
        rating_number = driver.find_element(By.XPATH, '//*[@id="module_product_review_star_1"]/div/a[1]').text
    except NoSuchElementException:
        rating_number = None

    try:
        qna_number = driver.find_element(By.XPATH, '//*[@id="module_product_review_star_1"]/div/a[2]').text
    except NoSuchElementException:
        qna_number = None

    try:
        discount = driver.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/div/span[2]').text
    except NoSuchElementException:
        discount = None

    try:
        bef_price = driver.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/div/span[1]').text
    except NoSuchElementException:
        bef_price = None

    return {
        'product_name': product_name,
        'brand_name': brand_name,
        'price': price,
        'seller': seller,
        'seller_rating': seller_rating,
        'seller_shipping': seller_shipping,
        'seller_chat': seller_chat,
        'rating_number': rating_number,
        'qna_number': qna_number,
        'discount': discount,
        'bef_price': bef_price
    }

In [7]:
def scrape_product_details(links, driver, search_keyword, pages):
    all_details = []
    
    for link in links:
        details = get_product_details(driver, link)
        all_details.append(details)
    
    driver.quit()
    
    search_keyword = search_keyword.replace(' ', '_')
    output_file = f"{search_keyword}_pages_{pages}_daraz_data.csv"
    
    keys = all_details[0].keys()
    with open(output_file, 'w', newline='', encoding='utf-8') as output_csv:
        dict_writer = csv.DictWriter(output_csv, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_details)
    
    print(f"Data has been saved to {output_file}")

In [8]:
search_keyword = input('Enter the search word: ')
page_number = int(input('Enter the page number: '))
all_links = get_product_links(search_keyword, page_number, chrome_driver)
scrape_product_details(all_links, chrome_driver, search_keyword, page_number)

Enter the search word: saree
Enter the page number: 3
Data has been saved to saree_pages_3_daraz_data.csv
